In [1]:

DB_ULR = "neo4j://localhost:7687"
DB_USER = "neo4j"
DB_PASS = "test1234"
DB_NAME = "opensanctions"

# Create driver
Also set the DB_NAME

In [39]:
import os
import pandas as pd
from graphdatascience import GraphDataScience # Python GDS client
import numpy as np
import threading
import multiprocessing

# And some cermony to create the driver and gds objects´
gds = GraphDataScience(DB_ULR, auth=(DB_USER, DB_PASS))
gds.set_database(DB_NAME)

In [12]:
gds.run_cypher(
    """ 
    CALL gds.graph.project('test','*',
        {
            HAS_IDENTIFIER: {orientation: 'UNDIRECTED'},
            ENTITY: {orientation: 'UNDIRECTED'},
            HAS_NAME: {orientation: 'UNDIRECTED'},
            ADDRESS_ENTITY: {orientation: 'UNDIRECTED'},
            UNKNOWN_LINK: {orientation: 'UNDIRECTED'},
            OWNERSHIP: {orientation: 'UNDIRECTED'},
            HAS_PHONE: {orientation: 'UNDIRECTED'},
            HAS_EMAIL: {orientation: 'UNDIRECTED'},
            OWNER: {orientation: 'UNDIRECTED'},
            DIRECTORSHIP: {orientation: 'UNDIRECTED'},
            REPRESENTATION: {orientation: 'UNDIRECTED'},
            HOLDER: {orientation: 'UNDIRECTED'},
            MEMBERSHIP: {orientation: 'UNDIRECTED'},
            ISSUER: {orientation: 'UNDIRECTED'},
            FAMILY: {orientation: 'UNDIRECTED'},
            ASSOCIATE: {orientation: 'UNDIRECTED'}
        }
    )
    """
)

,nodeProjection,relationshipProjection,graphName,nodeCount,relationshipCount,projectMillis
0,"{'__ALL__': {'label': '*', 'properties': {}}}",{'HAS_IDENTIFIER': {'orientation': 'UNDIRECTED...,test,2409145,7361736,283


In [40]:
G = gds.graph.get('test');

In [20]:
politicians = gds.run_cypher("MATCH (p:Politician) return id(p) as id")

In [22]:
politicians.head()

,id
0,3365985
1,3366214
2,3366282
3,3366356
4,3366460


In [42]:
nthreads = multiprocessing.cpu_count()
nthreads

10

In [43]:
chunks = np.array_split(politicians, nthreads)
all_threads = []

In [47]:
def worker(chunk, result):
    paths= gds.run_cypher(
        """ 
        MATCH (p:Politician) where id(p) in $ids
        with p
        call {
            with p
            CALL gds.allShortestPaths.dijkstra.stream('test', {
            sourceNode: p
            })
            YIELD index, targetNode, totalCost, path
            with index, totalCost, path, gds.util.asNode(targetNode) as target
            where target:SanctionedEntity
            RETURN
                index,
                target,
                totalCost,
                path
            LIMIT 1
        }
        return count(*)
        """,
        params = { 'ids' : chunk['id'].to_list()  }
    )
    result = pd.concat([result,paths], axis=0)

In [48]:
result = pd.DataFrame()
for chunk in chunks:
    thread = threading.Thread(target=worker, args=(chunk, result))
    all_threads.append(thread)
    thread.start()

for thread in all_threads:
    thread.join()

Exception in thread Thread-14:
Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniforge/base/envs/notebooks/lib/python3.9/threading.py", line 980, in _bootstrap_inner
    self.run()
  File "/opt/homebrew/Caskroom/miniforge/base/envs/notebooks/lib/python3.9/threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "/var/folders/71/lgz4464j18s5g22_s_lm696m0000gn/T/ipykernel_11106/23028501.py", line 25, in worker
  File "/opt/homebrew/Caskroom/miniforge/base/envs/notebooks/lib/python3.9/site-packages/pandas/core/generic.py", line 5583, in __getattr__
    return object.__getattribute__(self, name)
AttributeError: 'DataFrame' object has no attribute 'concat'
